# Figures present in the paper:

   - 1- Figures of Section 3
   - 2- Figures of Section 5
   - 3- Figures and tables of Section 4
   

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 12
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=12)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

# to delete this cell
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
from GPPY.spatial_windows import BallWindow, BoxWindow
from GPPY.point_processes import HomogeneousPoissonPointProcess
from GPPY.gravitational_force import force_k
from GPPY.gravity_point_process import GravityPointProcess
import copy
import numpy as np
from GPPY.monte_carlo_test_functions import (f_1, f_2, f_3, f_4, f_5, support_integrands)
import multiprocessing
from GPPY.monte_carlo_tests_setup import mc_results, mc_results_single_n

# 1- Figures of Section 3: 

## 1.1 Simulation of push

In [ ]:
d = 2
r=1
simu_window = BallWindow(center=[0]*d, radius=r)
intensity = 2000
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window, seed=1) 
gppy = GravityPointProcess(poisson_pp)
push_pp = gppy.pushed_point_pattern()


In [ ]:
#finding the force on each point
points = poisson_pp.points
nb_points = points.shape[0]
force_x, force_y=[],[]
epsilon = gppy.epsilon_critical
for k in range(0,nb_points):
    #force_ = epsilon*force_k(k=k, x=points[k], point_pattern=copy.deepcopy(poisson_pp))
    force = force_k(k=k, x=points[k], point_pattern=copy.deepcopy(poisson_pp))[0].tolist()
    force_x.append(force[0])
    force_y.append(force[1])


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
simu_window.plot(axis=ax[0], color="grey")
poisson_pp.plot(axis=ax[0], s=0.1)
poisson_pp.plot(axis=ax[1], c="g", s=0.1, label="Poisson")
ax[1].quiver(points[:,0], points[:,1], force_x, force_y, units='xy')
push_pp.plot(axis=ax[2], s=0.1)
simu_window.plot(axis=ax[2], color="grey")
ax[1].legend()
plt.savefig("poisson_and_push_and_force.pdf")
plt.show()

## 1.2 Variance reduction 

In [ ]:
d=3
nb_sample=50
nb_core = 4
nb_point_list=[500]
fct_list = [f_1, f_2, f_3, f_4]
fct_names = ["f_1", "f_2", "f_3", "f_4"]

support_window = support_integrands(d) 
simulation_window = BallWindow(center=[0]*d,
                               radius=3*np.sqrt(d)/2) #simulation window
intensity = nb_point_list[0]/support_window.volume
print(intensity)

In [ ]:
from GPPY.gravity_point_process import epsilon_critical

#Generate Poisson point process with fixed seeds
np.random.seed(122)
fixed_seed = np.random.poisson(lam=10, size=nb_sample).tolist()
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp_big = [poisson.generate_point_pattern(window=simulation_window, seed=s) for s in fixed_seed]
poisson_pp = [p.restrict_to_window(support_window) for p in poisson_pp_big]


#Variance for poisson
mc_poisson = mc_results_single_n(poisson_pp, "MC", 
                                 fct_list=fct_list, 
                                 fct_names=fct_names)

# variance for push
## f_1
std_f1_poisson = mc_poisson["mc_results_f_1"]["std_MC"]
## f_2
std_f2_poisson = mc_poisson["mc_results_f_2"]["std_MC"]
## f_3
std_f3_poisson = mc_poisson["mc_results_f_3"]["std_MC"]
## f_4
std_f4_poisson = mc_poisson["mc_results_f_4"]["std_MC"]

# epsilons
epsilon_0 = epsilon_critical(d, intensity)
print("d=", d, "intensity", intensity, ",Nb samples=", nb_sample, ",Epsilon critical=", epsilon_0)
epsilons = np.linspace(-3/2*epsilon_0, 2.5*epsilon_0, 25).tolist()
epsilons.append(epsilon_0)
epsilons.append(0)
epsilons.sort()
print(len(epsilons))

In [ ]:
stds_f1_push, stds_f3_push, stds_f2_push, stds_f4_push= [], [], [], []
#push
i=0
mean_nb_points = []
for epsilon in epsilons:
    print("---------------")
    print(i +1, "/ ", len(epsilons))
    time_start = time.time() 
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp = [GravityPointProcess(p) for p in poisson_pp_big]
        #using F
        push_pp_big = [g.pushed_point_pattern(epsilon=epsilon, core_number=nb_core) for g in gpp_pp]
    push_pp = [g.restrict_to_window(support_window) for g in push_pp_big]
    mc_push = mc_results_single_n(push_pp, "MCP", 
                                 fct_list=fct_list, 
                                 fct_names=fct_names)
    mean_nb_push_res = np.mean([s.points.shape[0] for s in push_pp])
    print("Mean N poisson=", np.mean([s.points.shape[0] for s in poisson_pp]),
          "Mean N push=", mean_nb_push_res)
    mean_nb_points.append(mean_nb_push_res)
    
    # variance for push
    ## f_1
    stds_f1_push.append(mc_push["mc_results_f_1"]["std_MCP"])
    ## f_2
    stds_f2_push.append(mc_push["mc_results_f_2"]["std_MCP"])
    ## f_3
    stds_f3_push.append(mc_push["mc_results_f_3"]["std_MCP"])
    ## f_4
    stds_f4_push.append(mc_push["mc_results_f_4"]["std_MCP"]) 
    i+=1
        


In [ ]:
fig, axis = plt.subplots(figsize=(4,3))
plt.plot(epsilons, mean_nb_points, 'b.')
plt.plot(epsilons, mean_nb_points, 'b')

plt.xlabel(r"$\varepsilon$")
plt.ylabel(r"$\bar{N}$")
plt.show()

In [ ]:
x = np.linspace(-3/5,3/5, 140)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_f_1 = f_1(points)
z_f_2 = f_2(points)
z_f_3 = f_3(points)
z_f_4 = f_5(points)
fig = plt.figure(figsize=(15, 4))
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_1, c=z_f_1, rasterized=True)
ax.set_title(r"$f_1$")
ax = fig.add_subplot(1, 3, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_2, c=z_f_2, rasterized=True)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_3, c=z_f_3, rasterized=True)
ax.set_title(r"$f_3$")
#ax.view_init(elev=40, azim=30)
# ax = fig.add_subplot(1, 4, 4, projection='3d')
# ax.scatter3D(X.ravel(), Y.ravel(), z_f_4, c=z_f_4, rasterized=True)
# ax.set_title(r"$f_4$")
# #ax.set_rasterization_zorder(0)
# ax.view_init(elev=40, azim=30)
plt.tight_layout()
plt.savefig("linear_stat_used_for_var_comp.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
#f_1
ax[0].plot(epsilons, stds_f1_push, "k.", label=r"Push" )
ax[0].plot(epsilons, stds_f1_push, "k" )
ax[0].plot(0, std_f1_poisson, "r*", label="Poisson")
ax[0].vlines(epsilon_0, ymin=min(stds_f1_push), ymax=max(stds_f1_push), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")
#ax[0].set_title(r"$f_1$")
ax[0].legend()
ax[0].set_xlabel(r"$\varepsilon$")
ax[0].set_ylabel(r"$\sigma$")
#f_2
ax[1].plot(epsilons, stds_f2_push,  "k.", label=r"Push")
ax[1].plot(epsilons, stds_f2_push,  "k")
ax[1].plot(0, std_f2_poisson, "r*", label="Poisson")
ax[1].vlines(epsilon_0, ymin=min(stds_f2_push), ymax=max(stds_f2_push), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")
ax[1].legend()
#ax[1].set_title(r"$f_2$")
ax[1].set_xlabel(r"$\varepsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
ax[2].plot(epsilons, stds_f3_push,  "k.", label=r"Push")
ax[2].plot(epsilons, stds_f3_push,  "k")
ax[2].plot(0, std_f3_poisson, "r*", label="Poisson")
ax[2].vlines(epsilon_0, ymin=min(stds_f3_push), ymax=max(stds_f3_push), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")

ax[2].legend()
#ax[2].set_title(r"$f_3$")
ax[2].set_xlabel(r"$\varepsilon$")
#ax[2].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
# ax[3].plot(epsilons, stds_f4_push,  "k.", label=r"Push")
# ax[3].plot(epsilons, stds_f4_push,  "k")
# ax[3].plot(0, std_f4_poisson, "r*", label="Poisson")
# ax[3].vlines(epsilon_0, ymin=min(stds_f4_push), ymax=max(stds_f4_push), colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")

# ax[3].legend()
# #ax[3].set_title(r"$f_3$")
# ax[3].set_xlabel(r"$\varepsilon$")
#ax[3].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
plt.tight_layout()
plt.savefig("variance_linear_stat_3d.pdf")
plt.show()

In [ ]:
import pickle

dic_std_poisson= {"std_f1_poisson": std_f1_poisson, 
                  "std_f2_poisson": std_f2_poisson,
                  "std_f3_poisson": std_f3_poisson,
                  "std_f4_poisson": std_f4_poisson
                  }
dic_std_push = {"std_f1_push": stds_f1_push, 
                  "std_f2_push": stds_f2_push,
                  "std_f3_push": stds_f3_push,
                  "std_f4_push": stds_f4_push
                  }
dict_result_var_epsilon= {"std_poisson": dic_std_poisson,
                          "std_push": dic_std_push}
with open('mc_std_3d_wrt_epsilon_paper.pickle', 'wb') as handle:
    pickle.dump(dict_result_var_epsilon, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
## 1.

# 2- Figures of Section 5

In [ ]:
from GPPY.spatial_windows import BallWindow
def sphere_number(p,q, d):
    r = (p-q)/2
    window = BallWindow(center= [0]*d, radius=q + r)
    return window.surface/(2*r**(d-1))
def next_points(a_0, p, q):
    r = (p-q)/2
    x_1 = (q**2-r**2 + 2*q*r)/(q+r)
    return np.array([[x_1, np.sqrt((q+r)**2 - x_1**2)], [x_1, - np.sqrt((q+r)**2 - x_1**2)]])
def next_points_2(a, p, q):
    r = (p-q)/2
    b_ = (4*r**2*a[1]**2 - 2*a[1]*(q+r)**2)
    a_ = a[1]**2 + a[0]**2 
    c_ = (q + r)**4 + 4*r**4 - 4*r**2*(q + r)**2 - a[0]**2*(q+r)**2
    delta = b_**2 - 4*a_*c_
    print(delta)
    x_2 = [(-b_-np.sqrt(delta))/(2*a_), (-b_+np.sqrt(delta))/(2*a_)]
    print(x_2)
    x_1 = [((q+r)**2 - 2*r**2 - x_2[0]*a[1])/a[1], ((q+r)**2 - 2*r**2 - x_2[1]*a[1])/a[1]]
    return np.array([[x_1[0], x_2[0]], [x_1[1], x_2[1]]]) 

In [ ]:
import numpy as np

epsilon = 0.5
x = np.array([3,76])
a_0 = [76, 0]
d=2
origin = [0]*d
norm_x = np.linalg.norm(x)
R=8
r=R/epsilon
p, q = norm_x + r, norm_x - r
sphere_nb = sphere_number(p=p, q=q, d=d)
points_next = next_points(a_0, p, q)
print(np.linalg.norm(points_next, axis=1))


In [ ]:

print("sphere number=", sphere_nb)
window0 = BallWindow(center=origin, radius=norm_x)
window1 = BallWindow(center=origin, radius=R)
window2 = BallWindow(center=origin, radius=norm_x-R/epsilon)
window3 = BallWindow(center=origin, radius=norm_x + R/epsilon)
window4 = BallWindow(center=x, radius=R/epsilon)
window5 = BallWindow(center=[np.sqrt((q+r)**2 - (q/(q+r))**2), q/(q+r)], radius=r)
window6 = BallWindow(center=points_next[0,:], radius=r)
window7 = BallWindow(center=points_next[1,:], radius=r)

In [ ]:
import numpy as np
from GPPY.spatial_windows import BallWindow

theta = np.arcsin(r/norm_x)
alpha = np.linspace(-theta, theta, 100)
u, v = (R+5)*np.cos(alpha), (R+5)*np.sin(alpha)
nb_theta = int(180/(theta*57.2958)/2)
print(nb_theta)

fig, ax = plt.subplots(figsize=(4,4))
window3.plot(axis=ax, color="b", linestyle="--")
window2.plot(axis=ax, color="b", linestyle="--")
ax.plot(0, 0, "k.")
plt.plot(a_0[0], a_0[1], 'r.')
plt.fill_between([norm_x-r,np.cos(theta)*(norm_x+r)], 
                 [np.sin(theta)*(norm_x-r), np.sin(theta)*(norm_x+r)], 
                 [-np.sin(theta)*(norm_x-r), -np.sin(theta)*(norm_x+r)], facecolor='green', interpolate=True)
y_ = np.linspace(-np.sin(theta)*(norm_x-r), np.sin(theta)*(norm_x-r))
plt.fill_betweenx(y_, np.sqrt((norm_x -r)**2 - y_**2), np.sqrt((norm_x + r)**2 - y_**2),
                  facecolor='green', interpolate=True)
y_ = np.linspace(-np.sin(theta)*(norm_x), np.sin(theta)*(norm_x))
plt.fill_betweenx(y_, np.sqrt((norm_x)**2 - y_**2), np.sqrt((norm_x + r)**2 - y_**2),
                  facecolor='green', interpolate=True)
for i in range(0, nb_theta ):
    a,b= np.cos(2*i*theta)*norm_x, np.sin(2*i*theta)*norm_x
    window = BallWindow(center=[a,b ], radius=r)
    window.plot(axis=ax, color="k")
    plt.plot(a, b, 'r.')
    window = BallWindow(center=[a,-b ], radius=r)
    window.plot(axis=ax, color="k")
    plt.plot(a, -b, 'r.')
    plt.plot([np.cos((2*i+1)*theta)*(norm_x-r), np.cos((2*i+1)*theta)*(norm_x+r)], 
             [np.sin((2*i+1)*theta)*(norm_x-r), np.sin((2*i+1)*theta)*(norm_x+r)], 'b' )
    plt.plot([np.cos((2*i+1)*theta)*(norm_x-r), np.cos((2*i+1)*theta)*(norm_x+r)], 
             [-np.sin((2*i+1)*theta)*(norm_x-r), -np.sin((2*i+1)*theta)*(norm_x+r)], 'b' )
c, d = np.cos(2*7*theta)*norm_x, np.sin(2*7*theta)*norm_x
window = BallWindow(center=[c,d ], radius=r)
window.plot(axis=ax, color="k")
plt.plot(c, d, 'r.')
plt.plot([0, c], [0, d], 'grey', linestyle="--")
plt.gca().annotate(r'$\|\mathbf{x}\|_2$', xy=(c/2, d -5), xycoords='data', fontsize=8, rotation=-10)
plt.plot([0,  np.cos(theta)*(norm_x+r)], 
        [0, np.sin(theta)*(norm_x+r)], 'b' )
plt.plot([0, np.cos(theta)*(norm_x+r)], 
        [0, -np.sin(theta)*(norm_x+r)], 'b' )
plt.plot([np.cos(15*theta)*(norm_x-r), np.cos(15*theta)*(norm_x+r)], 
             [np.sin((15)*theta)*(norm_x-r), np.sin((15)*theta)*(norm_x+r)], 'b' )
plt.plot([norm_x-r,norm_x],[0, 0], 'k' )
plt.gca().annotate(r'$f(\mathbf{x})$', xy=(norm_x -15, 3), xycoords='data', fontsize=8)
plt.plot([0,-8],[0, 0], 'k' )
plt.gca().annotate(r'$R$', xy=(-6, -6), xycoords='data', fontsize=7)
window1.plot(axis=ax, color="k")
window0.plot(axis=ax, color="grey", linestyle="-")
plt.plot(u,v, "grey")
plt.gca().annotate(r'$\theta$', xy=(R+7, -2), xycoords='data', fontsize=8)
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
plt.savefig("proof_existance_pic.pdf")
plt.show()


# 3- Figures of Section 4

In [ ]:
# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
plt.rc('axes', labelsize=12)
plt.rc('legend', fontsize=9)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

In [ ]:
import multiprocessing
from GPPY.monte_carlo_tests_setup import (plot_mc_results, mc_results, 
                                          dataframe_mse_results,
                                          dataframe_residual_test,
                                          dataframe_error_test,
                                          qq_plot_residual
                                         )
from GPPY.monte_carlo_test_functions import (f_1, f_2, f_3, f_4, f_5, f_6,
                                             exact_integral_f_1, exact_integral_f_2,
                                             exact_integral_f_3, exact_integral_f_4, 
                                             exact_integral_f_5,
                                             exact_integral_f_6,
                                             support_integrands)


## d=2

In [ ]:

import pickle
with open('data/mc_results_2d.pickle', 'rb') as file:
    mc_results_dict = pickle.load(file)
d =  mc_results_dict["d"]
mc_results_2d = mc_results_dict["mc_result"]
nb_point_2d = mc_results_dict["nb_point_list"]

In [ ]:
fct_list = [f_1, f_2, f_3]
fct_names = ["f_1", "f_2", "f_3"]
estimators = ["MC", 
              "MCP",
              "MCCV",
              "RQMC",
              "MCDPP"]
exact_integrals= [exact_integral_f_1(d), 
                  exact_integral_f_2(d),
                  exact_integral_f_3(d), 
                  ]
support_window = support_integrands(d) 

 Variance

In [ ]:
type_mc = ["MC", "MCP", "MCCV", "RQMC", "MCDPP"]
plot_mc_results(2, 
                mc_results_2d, 
                nb_point_2d, 
                fct_list=fct_list, 
                fct_names=fct_names,
                log_scale=False,
                type_mc=type_mc,
                save_fig="std_mc_2_d.pdf"
               )

In [ ]:
print("Normality test of residuals of the linear regression:")
dataframe_residual_test(mc_list=mc_results_2d, 
                        nb_point_list=nb_point_2d, 
                        fct_names=fct_names, 
                        test_type="SW")

In [ ]:
qq_plot_residual(mc_list=mc_results_2d, 
                        nb_point_list=nb_point_2d, 
                        fct_names=fct_names,
                #save_fig="qq_plot_2d.pdf"
                )

Error study

In [ ]:
plot_mc_results(d=2, 
                mc_list=mc_results_2d, 
                nb_point_list=nb_point_2d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="SE",
                plot_error=True,
                plot_std=False,
                log_scale=True,
                nb_subsample_nb_points=5,
                type_mc=type_mc,
                save_fig="square_error_mc_2d.pdf"
               )

In [ ]:
plot_mc_results(d=2, 
                mc_list=mc_results_2d, 
                nb_point_list=nb_point_2d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="Error",
                plot_error=True,
                plot_std=False,
                log_scale=False,
                nb_subsample_nb_points=5,
                save_fig="error_mc_2d.pdf"
               )

## d = 3

In [ ]:

import pickle
with open('data/mc_results_3d.pickle', 'rb') as file:
    mc_results_3d_dict = pickle.load(file)
d =  mc_results_3d_dict["d"]
mc_results_3d = mc_results_3d_dict["mc_result"]
nb_point_3d = mc_results_3d_dict["nb_point_list"]

exact_integrals= [exact_integral_f_1(d), 
                  exact_integral_f_2(d),
                  exact_integral_f_3(d),  
                  ]
support_window = support_integrands(d) 


variance

In [ ]:

plot_mc_results(3, 
                mc_results_3d, 
                nb_point_3d, 
                fct_list=fct_list, 
                fct_names=fct_names,
                log_scale=False, 
                type_mc=type_mc,
                save_fig="std_mc_3_d.pdf"
               )

In [ ]:
print("Normality test of residuals of the linear regression:")
dataframe_residual_test(mc_list=mc_results_3d, 
                        nb_point_list=nb_point_3d, 
                        fct_names=fct_names, 
                        test_type="SW")

In [ ]:
qq_plot_residual(mc_list=mc_results_3d, 
                 nb_point_list=nb_point_3d, 
                fct_names=fct_names,
                #save_fig="qq_plot_2d.pdf"
                )

Error

In [ ]:
plot_mc_results(d=3, 
                mc_list=mc_results_3d, 
                nb_point_list=nb_point_3d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="SE",
                plot_error=True,
                plot_std=False,
                log_scale=True,
                nb_subsample_nb_points=5,
                type_mc=type_mc,
                save_fig="square_error_mc_3d.pdf"
               )

In [ ]:
type_mc = ["MC", "MCP", "MCCV", "RQMC", "MCDPP"]

plot_mc_results(d=3, 
                mc_list=mc_results_3d, 
                nb_point_list=nb_point_3d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="Error",
                plot_error=True,
                plot_std=False,
                log_scale=False,
                nb_subsample_nb_points=5,
                type_mc=type_mc,
                #save_fig="square_error_mc_3d.pdf"
               )

In [ ]:
for name in fct_names[1:]:
    print("For "+ name + ":")
    df_error = dataframe_error_test(mc_results_3d, 
                                    nb_point_3d, 
                                    fct_name=name,
                                    type_mc_test="MCP")
    display(df_error)

## d=4

In [ ]:

import pickle
with open('mc_results_4d.pickle', 'rb') as file:
    mc_results_dict = pickle.load(file)
d =  mc_results_dict["d"]
mc_results_4d = mc_results_dict["mc_result"]
nb_point_4d = mc_results_dict["nb_point_list"]

fct_list = [f_1, f_2, f_3]
fct_names = ["f_1", "f_2", "f_3"]
estimators = ["MC", 
              "MCP",
              "MCCV",
              "RQMC",
              "MCDPP",]
exact_integrals= [exact_integral_f_1(d), 
                  exact_integral_f_2(d),
                  exact_integral_f_3(d), 
                  exact_integral_f_5(d), 
                  ]
support_window = support_integrands(d) 

In [ ]:

plot_mc_results(4, 
                mc_results_4d, 
                nb_point_4d, 
                fct_list=fct_list, 
                fct_names=fct_names,
                log_scale=False, 
                type_mc=type_mc[:-1],
                save_fig="std_mc_4_d.pdf"
               )

In [ ]:
print("Normality test of residuals of the linear regression:")
dataframe_residual_test(mc_list=mc_results_4d, 
                        nb_point_list=nb_point_4d, 
                        fct_names=fct_names, 
                        test_type="SW")

Error

In [ ]:
plot_mc_results(d=4, 
                mc_list=mc_results_4d, 
                nb_point_list=nb_point_4d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="SE",
                plot_error=True,
                plot_std=False,
                log_scale=True,
                nb_subsample_nb_points=5,
                type_mc=type_mc[:-1],
                save_fig="square_error_mc_4d.pdf"
               )

## d=5

In [ ]:

import pickle
with open('data/mc_results_5d.pickle', 'rb') as file:
    mc_results_dict = pickle.load(file)

d =  mc_results_dict["d"]
print("d=", d)
mc_results_5d = mc_results_dict["mc_result"]
nb_point_5d = mc_results_dict["nb_point_list"]
fct_list = [f_1, f_2, f_3]
fct_names = ["f_1", "f_2", "f_3"]
type_mc = ["MC", 
              "MCP",
              "MCCV",
              "RQMC",
              ]
exact_integrals= [exact_integral_f_1(d), 
                  exact_integral_f_2(d),
                  exact_integral_f_3(d),  
                  ]
support_window = support_integrands(d) 

In [ ]:

plot_mc_results(5, 
                mc_results_5d, 
                nb_point_5d, 
                fct_list=fct_list, 
                fct_names=fct_names,
                log_scale=False, 
                type_mc=type_mc,
                save_fig="std_mc_5_d.pdf"
               )

In [ ]:
print("Normality test of residuals of the linear regression:")
dataframe_residual_test(mc_list=mc_results_5d, 
                        nb_point_list=nb_point_5d, 
                        fct_names=fct_names, 
                        test_type="SW")

In [ ]:
plot_mc_results(d=5, 
                mc_list=mc_results_5d, 
                nb_point_list=nb_point_5d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="SE",
                plot_error=True,
                plot_std=False,
                log_scale=True,
                nb_subsample_nb_points=5,
                type_mc=type_mc,
                save_fig="square_error_mc_5d.pdf"
               )

In [ ]:
plot_mc_results(d=5, 
                mc_list=mc_results_5d, 
                nb_point_list=nb_point_5d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="Error",
                plot_error=True,
                plot_std=False,
                log_scale=False,
                nb_subsample_nb_points=5,
                type_mc=type_mc,
                #save_fig="square_error_mc_5d.pdf"
               )

 ## d=6

In [ ]:

import pickle
with open('data/mc_results_6d.pickle', 'rb') as file:
    mc_results_dict = pickle.load(file)

d =  mc_results_dict["d"]
print("d=", d)
mc_results_6d = mc_results_dict["mc_result"]
nb_point_6d = mc_results_dict["nb_point_list"]
fct_list = [f_1, f_2, f_3]
fct_names = ["f_1", "f_2", "f_3"]
type_mc = ["MC", 
              "MCP",
              "MCCV",
              "RQMC",
              ]
exact_integrals= [exact_integral_f_1(d), 
                  exact_integral_f_2(d),
                  exact_integral_f_3(d),  
                  ]
support_window = support_integrands(d) 

In [ ]:

plot_mc_results(6, 
                mc_results_6d, 
                nb_point_6d, 
                fct_list=fct_list, 
                fct_names=fct_names,
                log_scale=False, 
                type_mc=type_mc,
                save_fig="std_mc_6_d.pdf"
               )

In [ ]:
print("Normality test of residuals of the linear regression:")
dataframe_residual_test(mc_list=mc_results_6d, 
                        nb_point_list=nb_point_6d, 
                        fct_names=fct_names, 
                        test_type="SW")

In [ ]:
plot_mc_results(d=6, 
                mc_list=mc_results_6d, 
                nb_point_list=nb_point_6d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="SE",
                plot_error=True,
                plot_std=False,
                log_scale=True,
                nb_subsample_nb_points=5,
                type_mc=type_mc,
                save_fig="square_error_mc_6d.pdf"
               )

## d=7

In [ ]:

import pickle
with open('data/mc_results_7d.pickle', 'rb') as file:
    mc_results_dict = pickle.load(file)

d =  mc_results_dict["d"]
print("d=", d)
mc_results_7d = mc_results_dict["mc_result"]
nb_point_7d = mc_results_dict["nb_point_list"]
fct_list = [f_1, f_2, f_3]
fct_names = ["f_1", "f_2", "f_3"]
type_mc = ["MC", 
              "MCP",
              "MCCV",
              "RQMC",
              ]
exact_integrals= [exact_integral_f_1(d), 
                  exact_integral_f_2(d),
                  exact_integral_f_3(d),  
                  ]
support_window = support_integrands(d) 

In [ ]:
plot_mc_results(7, 
                mc_results_7d, 
                nb_point_7d, 
                fct_list=fct_list, 
                fct_names=fct_names,
                log_scale=False, 
                type_mc=type_mc,
                save_fig="std_mc_7_d.pdf"
               )

In [ ]:
print("Normality test of residuals of the linear regression:")
dataframe_residual_test(mc_list=mc_results_7d, 
                        nb_point_list=nb_point_7d, 
                        fct_names=fct_names, 
                        test_type="SW")

In [ ]:
plot_mc_results(d=7, 
                mc_list=mc_results_7d, 
                nb_point_list=nb_point_7d, 
                fct_list=fct_list[1:], 
                fct_names=fct_names[1:],
                error_type="SE",
                plot_error=True,
                plot_std=False,
                log_scale=True,
                nb_subsample_nb_points=5,
                type_mc=type_mc,
                save_fig="square_error_mc_7d.pdf"
               )

# 4- Figures of Section 6

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 12
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=12)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

# to delete this cell
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

## 4.1- Recursive Push

In [ ]:
d = 2
r=1
simu_window = BallWindow(center=[0]*d, radius=r)
intensity = 800
t_stop=50
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window, seed=1) 
gppy = GravityPointProcess(poisson_pp)
epsilon = gppy.epsilon_critical
push = gppy.pushed_point_pattern()
push_t = gppy.pushed_point_pattern(stop_time=t_stop)
push_inverse_t = gppy.pushed_point_pattern(epsilon=-epsilon, stop_time=t_stop)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(14, 4))
simu_window.plot(axis=ax[0], color="grey")
poisson_pp.plot(axis=ax[0], c="g", s=0.1, label=r"$t=0$")
push_inverse_t.plot(axis=ax[0], window=simu_window, c="b", s=0.1, label=r"$t=50, \varepsilon = -\varepsilon_0$")
ax[0].legend()
ax[0].set_xticks([])
ax[0].set_yticks([])
simu_window.plot(axis=ax[1], color="grey")
poisson_pp.plot(axis=ax[1], s=0.1)
poisson_pp.plot(axis=ax[1], c="g", s=0.1, label=r"$t=0$")
push.plot(axis=ax[1], c="b", s=0.1, label=r"$t=1, \varepsilon=\varepsilon_0$")
ax[1].legend()
ax[1].set_xticks([])
ax[1].set_yticks([])
simu_window.plot(axis=ax[2], color="grey")
poisson_pp.plot(axis=ax[2], c="g", s=0.1, label=r"$t=0$")
push_t.plot(axis=ax[2], c="b", s=0.1, label=r"$t=50, \varepsilon=\varepsilon_0$")
ax[2].legend()
ax[2].set_xticks([])
ax[2].set_yticks([])
plt.tight_layout()
plt.savefig("extreme_push.pdf")
plt.show()

## 4.2- Other model

In [ ]:
from GPPY.point_pattern import PointPattern
from GPPY.spatial_windows import BoxWindow, BallWindow
import structure_factor

def rescaled_point_pattern(point_pattern, window):
    if isinstance(window, BallWindow) or isinstance(window,structure_factor.spatial_windows.BallWindow):
        max_norm = window.radius
    else:
        max_norm = np.linalg.norm(window.bounds[0])
    max_norm_of_points = np.max(np.linalg.norm(point_pattern.points, axis=1))
    rescaled_points = point_pattern.points*max_norm/max_norm_of_points
    return PointPattern(rescaled_points, window=window)

def from_structure_factor_object_to_gppy(point_pattern):
    if isinstance(point_pattern.window, structure_factor.spatial_windows.BallWindow):
        window = BallWindow(center=point_pattern.window.center,
                    radius=point_pattern.window.radius)
    else:
        print( type(point_pattern.window))
        window = BoxWindow(bounds=point_pattern.window.bounds)
    return PointPattern(points=point_pattern.points, 
                        window=window,
                        intensity=point_pattern.intensity)

#### Pushed process

In [ ]:
import pickle

with open('data/list_samples_ginibre_pp.pkl', 'rb') as file:
    ginibre_pp_list = pickle.load(file)

with open('data/list_samples_thomas_pp.pkl', 'rb') as file:
    thomas_pp_list = pickle.load(file)



In [ ]:
observation_window = BallWindow(center=[0,0], radius=50)
#for Ginibre
ginibre_pp = from_structure_factor_object_to_gppy(ginibre_pp_list["ginibre_pp_ball"][0]).restrict_to_window(observation_window)
nb_core=4
if __name__ == "__main__":
    gpp_pp = GravityPointProcess(ginibre_pp) 
    #using F
    push_ginibre_pp = gpp_pp.pushed_point_pattern(core_number=nb_core)

#for Thomas
thomas_pp = from_structure_factor_object_to_gppy(thomas_pp_list["list_samples_thomas_pp_ball"][0]).restrict_to_window(observation_window)
if __name__ == "__main__":
    gpp_pp2 = GravityPointProcess(thomas_pp) 
    #using F
    push_thomas_pp = gpp_pp2.pushed_point_pattern(core_number=nb_core)

In [ ]:
from GPPY.numerical_integration import sobol_point_pattern
nb_points = ginibre_pp .points.shape[0]
sobol_pp = sobol_point_pattern(window=observation_window,
                                nb_points=nb_points,
                                seed=123)

if __name__ == "__main__":
    gpp_pp3 = GravityPointProcess(sobol_pp) 
    #using F
    push_sobol_pp = gpp_pp3.pushed_point_pattern(core_number=nb_core)

In [ ]:
print(ginibre_pp.points.shape[0], 
      thomas_pp.points.shape[0],
      sobol_pp.points.shape[0])

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(12, 8))
#Ginibre
observation_window.plot(axis=ax[0, 0], color="grey")
ginibre_pp.plot(axis=ax[0, 0], s=0.1, c="b")
ax[0, 0].set_title("Ginibre")
ax[0, 0].set_xticks([])
ax[0, 0].set_yticks([])
push_ginibre_pp.plot(axis=ax[1, 0], s=0.1)
observation_window.plot(axis=ax[1, 0], color="grey")
ax[1, 0].set_title("Pushed Ginibre")
ax[1, 0].set_xticks([])
ax[1, 0].set_yticks([])
#Sobol
observation_window.plot(axis=ax[0, 1], color="grey")
sobol_pp.plot(axis=ax[0, 1], s=0.1, c="b")
ax[0,1].set_title("Scrambled Sobol")
ax[0,1].set_xticks([])
ax[0,1].set_yticks([])
push_sobol_pp.plot(axis=ax[1,1], s=0.1)
observation_window.plot(axis=ax[1,1], color="grey")
ax[1,1].set_title("Pushed Scrambled Sobol")
ax[1,1].set_xticks([])
ax[1,1].set_yticks([])
#Thomas
observation_window.plot(axis=ax[0,2], color="grey")
thomas_pp.plot(axis=ax[0,2], s=0.1, c="b")
ax[0,2].set_title("Thomas")
ax[0,2].set_xticks([])
ax[0,2].set_yticks([])
push_thomas_pp.plot(axis=ax[1,2], s=0.1)
observation_window.plot(axis=ax[1,2], color="grey")
ax[1,2].set_title("Pushed Thomas")
ax[1,2].set_xticks([])
ax[1,2].set_yticks([])
plt.tight_layout()
plt.savefig("ginibre_and_thomas_push.pdf")
plt.show()

#### Universal variance reduction

In [ ]:
from GPPY.monte_carlo_test_functions import support_integrands
from GPPY.monte_carlo_tests_setup import mc_results_single_n
import math

In [ ]:
import pickle
with open('data/list_samples_ginibre_pp.pkl', 'rb') as file:
    ginibre_pp_list = pickle.load(file)

with open('data/list_samples_thomas_pp.pkl', 'rb') as file:
    thomas_pp_list = pickle.load(file)

### Ginibre

In [ ]:
from GPPY.gravity_point_process import epsilon_critical

d=2
support_window = support_integrands(d) 
simulation_window = BallWindow(center=[0]*d, radius=np.sqrt(d)/2)
#Big window used to restrit the Ginibre sample to it, to get intensity = 500
intensity=500
big_ball_window = BallWindow(center=[0]*d, radius=math.sqrt(intensity*simulation_window.volume))

fct_list = [f_1, f_2, f_3, f_4]
fct_names = ["f_1", "f_2", "f_3", "f_4"]
intensity=500
epsilon_0 = epsilon_critical(d, intensity)


#Ginibre
ginibre_pps = [from_structure_factor_object_to_gppy(g).restrict_to_window(big_ball_window) for g in ginibre_pp_list["ginibre_pp_ball"]]
ginibre_rescaled_pp = [rescaled_point_pattern(g, simulation_window) for g in ginibre_pps]

#Variance for ginibre
ginibre_pp_res = [g.restrict_to_window(support_window) for g in ginibre_rescaled_pp]
print(intensity, ginibre_pp_res[0].intensity)
mc_ginibre = mc_results_single_n(ginibre_pp_res, "MC", 
                                 fct_list=fct_list, 
                                 fct_names=fct_names)

# variance for push
## f_1
std_f1_ginibre = mc_ginibre["mc_results_f_1"]["std_MC"]
## f_2
std_f2_ginibre = mc_ginibre["mc_results_f_2"]["std_MC"]
## f_3
std_f3_ginibre = mc_ginibre["mc_results_f_3"]["std_MC"]

nb_sample = len(ginibre_pps)

epsilons = np.linspace(-3/2*epsilon_0, 3*epsilon_0, 40).tolist()
epsilons.append(epsilon_0)
epsilons.append(0)
epsilons.sort()
print(len(epsilons))
print("d=", d, "intensity=", intensity, ",Nb samples=", nb_sample, ",Epsilon critical=", epsilon_0)


In [ ]:
stds_f1_push_ginibre, stds_f3_push_ginibre, stds_f2_push_ginibre= [], [], []
mean_nb_point_ginibre = []
#push
i=0
for epsilon in epsilons:
    print("---------------")
    print(i +1, "/ ", len(epsilons))
    time_start = time.time() 
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp_ginibre = [GravityPointProcess(p) for p in ginibre_rescaled_pp]
        #using F
        push_pp_ginibre_big = [g.pushed_point_pattern(epsilon=epsilon, core_number=nb_core) for g in gpp_pp_ginibre]
    push_ginibre_pp = [g.restrict_to_window(support_window) for g in push_pp_ginibre_big]
    mean_nb_ginibre_res = np.mean([s.points.shape[0] for s in ginibre_pp_res])
    mean_nb_push_res = np.mean([s.points.shape[0] for s in push_ginibre_pp])
    mean_nb_point_ginibre.append(mean_nb_push_res)
    print("Mean N ginibre=", mean_nb_ginibre_res,  "Mean N push=", mean_nb_push_res)
    fig, axis = plt.subplots(1, 3, figsize=(6,2))
    ginibre_rescaled_pp[0].plot(axis=axis[0])
    simulation_window.plot(axis=axis[0], color="b")
    support_window.plot(axis=axis[0], color="r")
    axis[0].set_title("ginibre restricted")
    push_pp_ginibre_big[0].plot(axis=axis[1])
    support_window.plot(axis=axis[1], color="r")
    simulation_window.plot(axis=axis[1], color="b")
    axis[1].set_title("Big pushed ginibre")
    push_ginibre_pp[0].plot(axis=axis[2])
    support_window.plot(axis=axis[2], color="r")
    axis[2].set_title("Pushed ginibre restricted")
    plt.show()
    mc_push_ginibre = mc_results_single_n(push_ginibre_pp, "MCP", 
                                 fct_list=fct_list, 
                                 fct_names=fct_names)
    
    # variance for push
    ## f_1
    stds_f1_push_ginibre.append(mc_push_ginibre["mc_results_f_1"]["std_MCP"])
    ## f_2
    stds_f2_push_ginibre.append(mc_push_ginibre["mc_results_f_2"]["std_MCP"])
    ## f_3
    stds_f3_push_ginibre.append(mc_push_ginibre["mc_results_f_3"]["std_MCP"])
    i+=1
        


In [ ]:
fig, axis = plt.subplots(figsize=(4,3))
plt.plot(epsilons, mean_nb_point_ginibre, 'b.')
plt.plot(epsilons, mean_nb_point_ginibre, 'b')

plt.xlabel(r"$\varepsilon$")
plt.ylabel(r"$\bar{N}$")
plt.show()

In [ ]:
r=1 #radius observation window
x = np.atleast_2d(np.linspace(-r,r, 300)).T
z_f_1 = f_1(x)
z_f_2 = f_2(x)
z_f_3 = f_3(x)

fig = plt.figure(figsize=(3, 8))
ax = fig.add_subplot(3, 1, 1)
ax.plot(x, z_f_1 )
ax.set_title(r"$f_1$")
ax = fig.add_subplot(3, 1, 2)
ax.plot(x, z_f_2)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(3, 1, 3)
ax.plot(x,  z_f_3)
ax.set_title(r"$f_3$")
plt.tight_layout()
plt.savefig("linear_stat_used_for_var_comp_2d.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(4, 8))
#f_1
ax[0].plot(epsilons, stds_f1_push_ginibre, "k.", label=r"Push" )
ax[0].plot(epsilons, stds_f1_push_ginibre, "k" )
ax[0].plot(0, std_f1_ginibre, "r*", label="Ginibre")
ax[0].vlines(epsilon_0, ymin=min(stds_f1_push_ginibre), ymax=max(stds_f1_push_ginibre), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")
ax[0].set_title("Ginibre")
ax[0].legend()
ax[0].set_xlabel(r"$\varepsilon$")
ax[0].set_ylabel(r"$\sigma$")
#f_2
ax[1].plot(epsilons, stds_f2_push_ginibre,  "k.", label=r"Push")
ax[1].plot(epsilons, stds_f2_push_ginibre,  "k")
ax[1].plot(0, std_f2_ginibre, "r*", label="Ginibre")
ax[1].vlines(epsilon_0, ymin=min(stds_f2_push_ginibre), ymax=max(stds_f2_push_ginibre), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")
ax[1].legend()
#ax[1].set_title(r"$f_2$")
ax[1].set_xlabel(r"$\varepsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
ax[2].plot(epsilons, stds_f3_push_ginibre,  "k.", label=r"Push")
ax[2].plot(epsilons, stds_f3_push_ginibre,  "k")
ax[2].plot(0, std_f3_ginibre, "r*", label="Ginibre")
ax[2].vlines(epsilon_0, ymin=min(stds_f3_push_ginibre), ymax=max(stds_f3_push_ginibre), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")

ax[2].legend()
ax[2].set_xlabel(r"$\varepsilon$")
plt.tight_layout()
plt.savefig("variance_linear_stat_2d_ginibre.pdf")
plt.show()

### Thomas

In [ ]:

#Thomas
with open('data/list_samples_thomas_pp.pkl', 'rb') as file:
    thomas_pp_list = pickle.load(file)
#load data
thomas_pps = [from_structure_factor_object_to_gppy(g).restrict_to_window(big_ball_window) for g in thomas_pp_list["list_samples_thomas_pp_ball"]]
thomas_rescaled_pp = [rescaled_point_pattern(g, simulation_window) for g in thomas_pps]

#Variance for ginibre
thomas_pp_res = [g.restrict_to_window(support_window) for g in thomas_rescaled_pp]
mc_thomas = mc_results_single_n(thomas_pp_res, "MC", 
                                 fct_list=fct_list, 
                                 fct_names=fct_names)

# variance for push
## f_1
std_f1_thomas = mc_thomas["mc_results_f_1"]["std_MC"]
## f_2
std_f2_thomas = mc_thomas["mc_results_f_2"]["std_MC"]
## f_3
std_f3_thomas = mc_thomas["mc_results_f_3"]["std_MC"]


stds_f1_push_thomas, stds_f3_push_thomas, stds_f2_push_thomas, mean_nb_push_thomas= [], [], [], []
#push
nb_core=3
i=0
for epsilon in epsilons:
    print("---------------")
    print(i +1, "/ ", len(epsilons))
    time_start = time.time() 
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp_thomas = [GravityPointProcess(p) for p in thomas_rescaled_pp]
        #using F
        push_pp_thomas_big = [g.pushed_point_pattern(epsilon=epsilon, core_number=nb_core) for g in gpp_pp_thomas]
    push_thomas_pp = [g.restrict_to_window(support_window) for g in push_pp_thomas_big]
    mean_nb_thomas_res = np.mean([s.points.shape[0] for s in thomas_pp_res])
    mean_nb_push_res = np.mean([s.points.shape[0] for s in push_thomas_pp])
    print("Mean N thomas=", mean_nb_thomas_res,  "Mean N push=", mean_nb_push_res)
    mean_nb_push_thomas.append(mean_nb_push_res)
    fig, axis = plt.subplots(1, 3, figsize=(6,2))
    thomas_rescaled_pp[0].plot(axis=axis[0])
    simulation_window.plot(axis=axis[0], color="b")
    support_window.plot(axis=axis[0], color="r")
    axis[0].set_title("thomas restricted")
    push_pp_thomas_big[0].plot(axis=axis[1])
    support_window.plot(axis=axis[1], color="r")
    simulation_window.plot(axis=axis[1], color="b")
    axis[1].set_title("Big pushed thomas")
    push_thomas_pp[0].plot(axis=axis[2])
    support_window.plot(axis=axis[2], color="r")
    axis[2].set_title("Pushed thomas restricted")
    plt.show()
    mc_push_thomas = mc_results_single_n(push_thomas_pp, "MCP", 
                                 fct_list=fct_list, 
                                 fct_names=fct_names)
    
    # variance for push
    ## f_1
    stds_f1_push_thomas.append(mc_push_thomas["mc_results_f_1"]["std_MCP"])
    ## f_2
    stds_f2_push_thomas.append(mc_push_thomas["mc_results_f_2"]["std_MCP"])
    ## f_3
    stds_f3_push_thomas.append(mc_push_thomas["mc_results_f_3"]["std_MCP"])
    i+=1
        


In [ ]:
fig = plt.subplots(figsize=(4,3))
plt.plot(epsilons, mean_nb_push_thomas, 'b.')
plt.plot(epsilons, mean_nb_push_thomas, 'b')
plt.xlabel(r"$\varepsilon$")
plt.ylabel(r"$\bar{N}$")
plt.show()

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(4, 8))
#f_1
ax[0].plot(epsilons, stds_f1_push_thomas, "k.", label=r"Push" )
ax[0].plot(epsilons, stds_f1_push_thomas, "k" )
ax[0].plot(0, std_f1_thomas, "r*", label="Thomas")
ax[0].vlines(epsilon_0, ymin=min(stds_f1_push_thomas), ymax=max(stds_f1_push_thomas), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")
ax[0].set_title("Thomas")
ax[0].legend()
ax[0].set_xlabel(r"$\varepsilon$")
ax[0].set_ylabel(r"$\sigma$")
#f_2
ax[1].plot(epsilons, stds_f2_push_thomas,  "k.", label=r"Push")
ax[1].plot(epsilons, stds_f2_push_thomas,  "k")
ax[1].plot(0, std_f2_thomas, "r*", label="Thomas")
ax[1].vlines(epsilon_0, ymin=min(stds_f2_push_thomas), ymax=max(stds_f2_push_thomas), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")
ax[1].legend()
#ax[1].set_title(r"$f_2$")
ax[1].set_xlabel(r"$\varepsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
ax[2].plot(epsilons, stds_f3_push_thomas,  "k.", label=r"Push")
ax[2].plot(epsilons, stds_f3_push_thomas,  "k")
ax[2].plot(0, std_f3_thomas, "r*", label="Thomas")
ax[2].vlines(epsilon_0, ymin=min(stds_f3_push_thomas), ymax=max(stds_f3_push_thomas), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")
ax[2].legend()
ax[2].set_xlabel(r"$\varepsilon$")
plt.tight_layout()
plt.savefig("variance_linear_stat_2d_thomas.pdf")
plt.show()

### Sobol

In [ ]:
d=2
simulation_window = BallWindow(center=[0]*d, radius=np.sqrt(d)/2)
intensity = 500
nb_points = int(intensity*simulation_window.volume)
nb_sample = len(ginibre_pp_list["ginibre_pp_ball"])
np.random.seed(42)  # Set the seed value for reproducibility
seed_list = np.random.poisson(lam=100, size=nb_sample).tolist()
#Sobol
fct_list = [f_1, f_2, f_3]
fct_names = ["f_1", "f_2", "f_3"]
sobol_pp = [sobol_point_pattern(window=simulation_window,
                                nb_points=nb_points, seed=s) for s in seed_list]

support_window = support_integrands(d) 
#Variance for sobol
sobol_pp_res = [s.restrict_to_window(support_window) for s in sobol_pp]
mc_sobol = mc_results_single_n(sobol_pp_res, "MC", 
                                 fct_list=fct_list, 
                                 fct_names=fct_names)

# variance for push
## f_1
std_f1_sobol = mc_sobol["mc_results_f_1"]["std_MC"]
## f_2
std_f2_sobol = mc_sobol["mc_results_f_2"]["std_MC"]
## f_3
std_f3_sobol = mc_sobol["mc_results_f_3"]["std_MC"]


In [ ]:
from GPPY.gravity_point_process import epsilon_critical

epsilon_0 = epsilon_critical(d, intensity)
print("d=", d, "intensity=", intensity, ",Nb samples=", nb_sample, ",Epsilon critical=", epsilon_0)
epsilons = np.linspace(-3/2*epsilon_0, 3*epsilon_0, 40).tolist()
epsilons.append(epsilon_0)
epsilons.append(0)
epsilons.sort()
print(len(epsilons))
print("number of points to be used")


In [ ]:

stds_f1_push_sobol, stds_f3_push_sobol, stds_f2_push_sobol= [], [], []
#push
i=0
for epsilon in epsilons:
    print("---------------")
    print(i +1, "/ ", len(epsilons))
    time_start = time.time() 
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp_sobol = [GravityPointProcess(p) for p in sobol_pp]
        #using F
        push_pp_sobol_big = [s.pushed_point_pattern(epsilon=epsilon, core_number=nb_core) for s in gpp_pp_sobol]
    push_sobol_pp = [g.restrict_to_window(support_window) for g in push_pp_sobol_big]
    mean_nb_sobol_res = np.mean([s.points.shape[0] for s in sobol_pp_res])
    mean_nb_push_res = np.mean([s.points.shape[0] for s in push_sobol_pp])
    print("Mean N sobol=", mean_nb_sobol_res,  "Mean N push=", mean_nb_push_res)
    fig, axis = plt.subplots(1, 3, figsize=(4,2))
    sobol_pp_res[0].plot(axis=axis[0])
    support_window.plot(axis=axis[0], color="r")
    axis[0].title("Sobol restricted")
    push_pp_sobol_big[0].plot(axis=axis[1])
    support_window.plot(axis=axis[1], color="r")
    axis[1].title("Big pushed Sobol")
    push_sobol_pp[0].plot(axis=axis[2])
    support_window.plot(axis=axis[2], color="r")
    axis[2].title("Pushed Sobol restricted")
    plt.show()
    mc_push_sobol = mc_results_single_n(push_sobol_pp, 
                                        "MCP", 
                                        fct_list=fct_list, 
                                        fct_names=fct_names)
    
    # variance for push
    ## f_1
    stds_f1_push_sobol.append(mc_push_sobol["mc_results_f_1"]["std_MCP"])
    ## f_2
    stds_f2_push_sobol.append(mc_push_sobol["mc_results_f_2"]["std_MCP"])
    ## f_3
    stds_f3_push_sobol.append(mc_push_sobol["mc_results_f_3"]["std_MCP"])
    i+=1
        


In [ ]:
fig, axis = plt.subplots(figsize=(4,3))
plt.plot(epsilons, mean_nb_point_ginibre, 'b.')
plt.xlabel(r"$\varepsilon$")
plt.ylabel(r"$\bar{N}$")
plt.show()

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(4, 8))
#f_1
ax[0].plot(epsilons, stds_f1_push_sobol, "k.", label=r"Push" )
ax[0].plot(epsilons, stds_f1_push_sobol, "k" )
ax[0].plot(0, std_f1_sobol, "r*", label="Scrambled Sobol")
ax[0].vlines(epsilon_0, ymin=min(stds_f1_push_sobol), ymax=max(stds_f1_push_sobol), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")
ax[0].set_title("Scrambled Sobol")
ax[0].legend()
ax[0].set_xlabel(r"$\varepsilon$")
ax[0].set_ylabel(r"$\sigma$")
#f_2
ax[1].plot(epsilons, stds_f2_push_sobol,  "k.", label=r"Push")
ax[1].plot(epsilons, stds_f2_push_sobol,  "k")
ax[1].plot(0, std_f2_sobol, "r*", label="Scrambled Sobol")
ax[1].vlines(epsilon_0, ymin=min(stds_f2_push_sobol), ymax=max(stds_f2_push_sobol), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")
ax[1].legend()
#ax[1].set_title(r"$f_2$")
ax[1].set_xlabel(r"$\varepsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
ax[2].plot(epsilons, stds_f3_push_sobol,  "k.", label=r"Push")
ax[2].plot(epsilons, stds_f3_push_sobol,  "k")
ax[2].plot(0, std_f3_sobol, "r*", label="Scrambled Sobol")
ax[2].vlines(epsilon_0, ymin=min(stds_f3_push_sobol), ymax=max(stds_f3_push_sobol), 
             colors="grey", linestyles='dashed', label=r"$\varepsilon_0$")

ax[2].legend()
ax[2].set_xlabel(r"$\varepsilon$")
plt.tight_layout()
plt.savefig("variance_linear_stat_2d_sobol.pdf")
plt.show()

In [ ]:
len()